### Import Dependencies


In [7]:
import json
import cv2
import numpy as np
from ultralytics import YOLO
from shapely.geometry import Polygon, box


### Load Slot Annotations (VGG or COCO)


In [9]:
ANNOTATION_W = 1919
ANNOTATION_H = 1079

VIDEO_W = 1280
VIDEO_H = 720

scale_x = VIDEO_W / ANNOTATION_W
scale_y = VIDEO_H / ANNOTATION_H

print("scale_x:", scale_x)
print("scale_y:", scale_y)


scale_x: 0.6670140698280355
scale_y: 0.6672845227062094


In [10]:
def load_scaled_slots_vgg(json_path, sx, sy):
    with open(json_path) as f:
        data = json.load(f)

    slots = []

    for img_data in data.values():
        for region in img_data["regions"].values():
            xs = region["shape_attributes"]["all_points_x"]
            ys = region["shape_attributes"]["all_points_y"]

            scaled_pts = [(x * sx, y * sy) for x, y in zip(xs, ys)]

            slots.append({
                "slot_id": region["region_attributes"]["label"],
                "polygon": Polygon(scaled_pts)
            })

    return slots



In [11]:
slots = load_scaled_slots_vgg("../data/labels/vgg.json",scale_x,scale_y)

print("Total slots:", len(slots))


Total slots: 5


### Load YOLO Car Detector


In [12]:
model = YOLO("../models/yolov8n.pt")

In [6]:
img = cv2.imread("../data/image/image.png")

results = model(img, conf=0.3)[0]



0: 384x640 10 cars, 64.6ms
Speed: 38.6ms preprocess, 64.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


In [2]:
import sys, os
sys.path.append(os.path.abspath(".."))

In [3]:
from config.firebase_init import db

def init_slots(slot_ids):
    for slot_id in slot_ids:
        db.collection("slots").document(f"slot_{slot_id}").set({
            "occupied": False
        })

# Example: slots 1 to 10
init_slots([1,2,3,4,5,6,7,8,9,10])

print("Slots initialized in Firebase")


Slots initialized in Firebase


In [4]:
from datetime import datetime
import time

In [5]:
def push_slot_status_to_firebase(slot_status):
    for slot_id, occupied in slot_status.items():
        db.collection("slots").document(f"slot_{slot_id}").set({
            "occupied": occupied,
            "last_updated": datetime.now().strftime("%d-%m-%Y %H:%M:%S")
        }, merge=True)


In [13]:
cap = cv2.VideoCapture("../data/video/CCTV_Video_of_Multiple_Vehicles.mp4")
assert cap.isOpened(), "Video not opened"

last_firebase_push = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (VIDEO_W, VIDEO_H))

    results = model(frame, conf=0.4, verbose=False)[0]

    vehicle_boxes = []
    for b in results.boxes.data:
        x1, y1, x2, y2, conf, cls = b.tolist()
        if int(cls) in [2, 3, 5, 7]:
            vehicle_boxes.append(box(x1, y1, x2, y2))

    # 🔴 NEW: store slot availability
    slot_status = {}

    for slot in slots:
        occupied = False
        for vbox in vehicle_boxes:
            if slot["polygon"].intersection(vbox).area / slot["polygon"].area > 0.15:
                occupied = True
                break

        slot_status[slot["slot_id"]] = occupied  # ✅ KEY LINE

        pts = np.array(slot["polygon"].exterior.coords, np.int32)
        color = (0, 0, 255) if occupied else (0, 255, 0)

        cv2.polylines(frame, [pts], True, color, 2)
        cv2.putText(
            frame,
            f"Slot {slot['slot_id']}",
            tuple(pts[0]),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            color,
            2
        )

    # 🔴 NEW: push to Firebase every 2 seconds (NOT every frame)
    if time.time() - last_firebase_push > 2:
        push_slot_status_to_firebase(slot_status)
        last_firebase_push = time.time()

    cv2.imshow("Smart Parking CCTV", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [17]:
from services.ticket_service import generate_ticket

ticket = generate_ticket()
print(ticket)


{'ticket_id': '4382fae9-b580-453e-b7cc-b5b0e1cfc705', 'slot_id': 'slot_10', 'in_time': '18-01-2026 02:01:25'}
